In [30]:
#!pip install selenium
#!apt-get -q update # to update ubuntu to correctly run apt install
#!apt install -yq chromium-chromedriver
!cp /Users/DRNPRO/Documents/chromedriver

import sys
sys.path.insert(0,'/Users/DRNPRO/Documents/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

usage: cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file ... target_directory


In [32]:
def altElement(l):
    return l[::2]

# removes extra data that comes with each comment
def split_comment(s):
    splitted = s.split('\n')
    comment_body = splitted[5:-2]
    return comment_body

# converts comments to a string and removes brackets
def list_to_string(x):
    string = str(x)
    return string.rstrip(']').lstrip('[')

In [33]:
import pandas as pd
driver = webdriver.Chrome('/Users/DRNPRO/Documents/chromedriver', options=chrome_options)
driver.get('https://www.beeradvocate.com/beer/top-rated/')
urls = []
beer_names = []

# Get list of beer urls
url_paths = driver.find_elements_by_xpath('//*[contains(@href, "/beer/profile/")]')
for i in url_paths:
    urls.append(i.get_attribute('href'))
beer_urls = altElement(urls)

# Get list of beer names
beer_name = driver.find_elements_by_xpath('//*[contains(@href, "/beer/profile/")]')
for i in beer_name:
    beer_names.append(i.text)
beer_names = altElement(beer_names)

In [ ]:
# scrape comments for each beer
beer = pd.DataFrame(columns = ['product_name','product_review','user_rating'])


for beer_page in beer_urls:
    driver.get(beer_page)
    ids = driver.find_elements_by_xpath("//*[contains(@id,'rating_fullview_container')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('ba-user'))

    for x in comment_ids:
        
        product_name = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div[3]/div/div/div[1]/h1')[0]
        product = product_name.text.split('\n')[0]

        user_message = driver.find_elements_by_xpath('//*[@ba-user= ' + x + ']')[0]
        comment = user_message.text

        user_rating = driver.find_elements_by_xpath('//*[@ba-user= ' + x + ']//*[@id="rating_fullview_content_2"]/span[2]')[0]
        rating = user_rating.text

        beer.loc[len(beer)] = [product,comment,rating]

beer['product_review'] = beer['product_review'].map(split_comment)
beer['product_review'] = beer['product_review'].map(list_to_string)

beer.to_csv('beer.csv')
print("Beer data exported to beer.csv :)")

In [ ]:
df = pd.read_csv('beer.csv')

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

In [ ]:
reviews = df.product_review
reviews.head()

In [ ]:
# Attributes to look for
original_atts = ['complex', 'spicy', 'aroma']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.parsing.preprocessing import remove_stopwords

import string
from string import punctuation
import translate